In [1]:
import numpy as np
import pandas as pd
import warnings; warnings.simplefilter('ignore')

In [2]:
cd ml-100k/

/Users/ayalahlou/Final Project 001/ml-100k


In [3]:
names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('u.data', sep='\t', names=names)

## COLLABORATIVE FILTERING

Collaborative filtering can be divided into 2 categories:
    * Memory Based Approach: this technique find similar users based on cosine similarity and pearson correlation and take weighted average of ratings.
    
    * Model Based Approach: this technique uses Machine LEarning to find user ratings of unrated items. 

In this part of our final project, we will be using Memory Based Collaborative Filtering, more precisely user-item filtering as we are interested in taking a particular user, find users that are similar to that user based on similarity of ratings, and recommend movies that those similar users liked. 

To get started, we will first build the User-Movie matrix.

Whenever a User X has rated a movie Y, the corresponding cell would have the value of the rating User X gave. If User X did not rate a movie Z, the corresponding cell would be filled as 0. However, we shouldn't consider the missing ratings to be truly zero but rather just as empty entries. 

In [4]:
n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]
ratings = np.zeros((n_users, n_items))
for row in df.itertuples():
    ratings[row[1]-1, row[2]-1] = row[3]

Now, we shall split our data into a training set and a testing set by removing 10 ratings per user from the training set and placing them in the test set.

In [5]:
def split(ratings):
    test = np.zeros(ratings.shape)
    train = ratings.copy()
    for user in range(ratings.shape[0]):
        test_ratings = np.random.choice(ratings[user, :].nonzero()[0], size=10, replace=False)
        train[user, test_ratings] = 0.
        test[user, test_ratings] = ratings[user, test_ratings]
        
    # Test and training are truly disjoint
    assert(np.all((train * test) == 0)) 
    return train, test
train, test = split(ratings)

## COSINE SIMILARITY

A common distance metric of user-item similarity is cosine similarity. The metric can be thought of geometrically if one treats a given user's row of the ratings matrix as a vector. For user-based collaborative filtering, two users' similarity is measured as the cosine of the angle between the two users' vectors. For users ${u}$ and ${u'}$, the cosine similarity is

$$ sim(u, u') = cos(\theta{}) = \frac{\textbf{r}_{u} \dot{} \textbf{r}_{u'}}{\| \textbf{r}_{u} \| \| \textbf{r}_{u'} \|} = \sum_{i} \frac{r_{ui}r_{u'i}}{\sqrt{\sum\limits_{i} r_{ui}^2} \sqrt{\sum\limits_{i} r_{u'i}^2} } $$

The cosine similarity will range from 0 to 1 because there are no negative ratings. 

In [6]:
def similarityfunc(ratings, kind='user', epsilon=1e-9):
    # epsilon -> small number for handling dived-by-zero errors
    if kind == 'user':
        sim = ratings.dot(ratings.T) + epsilon
    elif kind == 'item':
        sim = ratings.T.dot(ratings) + epsilon
    norms = np.array([np.sqrt(np.diagonal(sim))])
    return (sim / norms / norms.T)

In [7]:
user_similarity = similarityfunc(train, kind='user')
item_similarity = similarityfunc(train, kind='item')

Now that we have the User-Movie Matrix, we can predict the ratings that were not included with the data. 

Using these predictions, we can then compare them with the test data to eavluate to validate the quality of our recommender model.

For user-based collaborative filtering, we predict that a user's $u$'s rating for item $i$ is given by the sum of all other users' ratings for the same item $i$ where the weighting is the cosine similarity between the each user and the input user $u$.

$$\hat{r}_{ui} = \sum\limits_{u'}sim(u, u') r_{u'i}$$
We must also normalize by the number of ${r_{u'i}}$ ratings:
$$\hat{r}_{ui} = \frac{\sum\limits_{u'} sim(u, u') r_{u'i}}{\sum\limits_{u'}|sim(u, u')|}$$
As with before, our computational speed will benefit greatly by favoring NumPy functions over for loops. With our slow function below, even though I use NumPy methods, the presence of the for-loop still slows the algorithm

In [8]:
def predict(ratings, similarity, kind='user'):
    if kind == 'user':
        return similarity.dot(ratings) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif kind == 'item':
        return ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])

We’ll use the scikit-learn’s mean squared error function as our validation metric. Comparing user- and item-based collaborative filtering, it looks like user-based collaborative filtering gives us a better result.

In [9]:
from sklearn.metrics import mean_squared_error

def get_mse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

In [10]:
item_prediction = predict(train, item_similarity, kind='item')
user_prediction = predict(train, user_similarity, kind='user')

print ('User-based CF MSE: ' + str(get_mse(user_prediction, test)))
print ('Item-based CF MSE: ' + str(get_mse(item_prediction, test)))

User-based CF MSE: 8.453780443912272
Item-based CF MSE: 11.568891782031098


Top-$k$ Collaborative Filtering

We can attempt to improve our prediction MSE by only considering the top $k$ users who are most similar to the input user (or, similarly, the top $k$ items). That is, when we calculate the sums over $u'$
$$\hat{r}_{ui} = \frac{\sum\limits_{u'} sim(u, u') r_{u'i}}{\sum\limits_{u'}|sim(u, u')|}$$
we only sum over the top $k$ most similar users. A slow implementation of this algorithm is shown below. While I am sure that there is a way to use numpy sorting to get rid of the double for-loops, I got pretty frustrated desciphering the 2D argsort output and just settled for the slow loop.
As is shown below, employing this method actually halves our error!

In [11]:
def predict_topk(ratings, similarity, kind='user', k=40):
    pred = np.zeros(ratings.shape)
    if kind == 'user':
        for i in range(ratings.shape[0]):
            top_k_users = [np.argsort(similarity[:,i])[:-k-1:-1]]
            for j in range(ratings.shape[1]):
                pred[i, j] = similarity[i, :][top_k_users].dot(ratings[:, j][top_k_users]) 
                pred[i, j] /= np.sum(np.abs(similarity[i, :][top_k_users]))
    if kind == 'item':
        for j in range(ratings.shape[1]):
            top_k_items = [np.argsort(similarity[:,j])[:-k-1:-1]]
            for i in range(ratings.shape[0]):
                pred[i, j] = similarity[j, :][top_k_items].dot(ratings[i, :][top_k_items].T) 
                pred[i, j] /= np.sum(np.abs(similarity[j, :][top_k_items]))        
    
    return pred

In [12]:
'''pred = predict_topk(train, user_similarity, kind='user', k=40)
print ('Top-k User-based CF MSE: ' + str(get_mse(pred, test)))

pred = predict_topk(train, item_similarity, kind='item', k=40)
print ('Top-k Item-based CF MSE: ' + str(get_mse(pred, test)))'''

"pred = predict_topk(train, user_similarity, kind='user', k=40)\nprint ('Top-k User-based CF MSE: ' + str(get_mse(pred, test)))\n\npred = predict_topk(train, item_similarity, kind='item', k=40)\nprint ('Top-k Item-based CF MSE: ' + str(get_mse(pred, test)))"

In [13]:
idx_to_movie = {}
a=[]
with open('u.item', 'r',errors='replace') as f:
    for line in f.readlines():
        info = line.split('|')
        info[1]= info[1].split(" (")[0]
        idx_to_movie[int(info[0])-1] = info[1]
        a.append(info)
def top_k_movies(similarity, mapper, movie_idx, k=10):
    return [mapper[x] for x in np.argsort(similarity[movie_idx,:])[:-k-1:-1]]

In [14]:
fav='Toy Story'
for i in range(1682):
    if a[i][1]==fav:
        idx=i
        break
movies = top_k_movies(item_similarity, idx_to_movie, idx)
display(movies)

['Toy Story',
 'Star Wars',
 'Return of the Jedi',
 'Independence Day',
 'Raiders of the Lost Ark',
 'Rock, The',
 'Star Trek: First Contact',
 'Mission: Impossible',
 'Willy Wonka and the Chocolate Factory',
 'Empire Strikes Back, The']

In [15]:
li=[]
try:
    
    for j in range(len(movies)):
        for i in range(len(mov)):
            #if movies[j]=='Batman':
                #reclink='https://www.imdb.com/title/'+'tt0096895'+'/?ref_=fn_al_tt_1'
                #li.append(reclink)
                #break
            if movies[j]==mov['original_title'][i]:
                reclink='https://www.imdb.com/title/'+mov['imdb_id'][i]+'/?ref_=fn_al_tt_1'
                li.append(reclink)
                break
            elif movies[j]!=mov['original_title'][i] and i==45465:
                reclink='(link not available)'
                li.append(reclink)
                break   
    for o in range(10):
        if o==0: 
            print ('Users who liked', movies[o],'also liked: \n ' )
        else:
            print('*',movies[o], li[o])
except:
    print(movies[i], '(link not available)')

Toy Story (link not available)


## PEARSON CORRELATION

One thing that could be the issue is that very popular movies like Star Wars are being favored. We can remove some of this bias by considering a different similarity metric - the pearson correlation. I’ll just grab the built-in scikit-learn function for computing this.

In [16]:
from sklearn.metrics import pairwise_distances

item_correlation = 1 - pairwise_distances(train.T, metric='correlation')
item_correlation[np.isnan(item_correlation)] = 0.

In [17]:

idx_to_movie = {}
a=[]
with open('u.item', 'r',errors='replace') as f:
    for line in f.readlines():
        info = line.split('|')
        info[1]= info[1].split(" (")[0]
        idx_to_movie[int(info[0])-1] = info[1]
        a.append(info)
    
def top_k_movies(similarity, mapper, movie_idx, k=10):
    return [mapper[x] for x in np.argsort(similarity[movie_idx,:])[:-k-1:-1]]

In [21]:
fav='Snow White and the Seven Dwarfs'
for i in range(1682):
    if a[i][1]==fav:
        idx=i
        break
movies = top_k_movies(item_correlation, idx_to_movie, idx)
display(movies)
i

['Snow White and the Seven Dwarfs',
 'Cinderella',
 'Pinocchio',
 'Beauty and the Beast',
 'Dumbo',
 'Fantasia',
 'Mary Poppins',
 'Aladdin',
 'Lion King, The',
 'Alice in Wonderland']

98

In [22]:
mov=pd.read_csv('movies_metadata.csv')
mov=mov.loc[:,'imdb_id':'original_title']
mov=mov.drop(columns=['original_language'])

In [23]:

li=[]
try:
    
    for j in range(len(movies)):
        for i in range(len(mov)):
            #if movies[j]=='Batman':
                #reclink='https://www.imdb.com/title/'+'tt0096895'+'/?ref_=fn_al_tt_1'
                #li.append(reclink)
                #break
            if movies[j]==mov['original_title'][i]:
                reclink='https://www.imdb.com/title/'+mov['imdb_id'][i]+'/?ref_=fn_al_tt_1'
                li.append(reclink)
                break
            elif movies[j]!=mov['original_title'][i] and i==45465:
                reclink='(link not available)'
                li.append(reclink)
                break   
    for o in range(10):
        if o==0: 
            print ('Users who liked', movies[o],'also liked: \n ' )
        else:
            print('*',movies[o], li[o])
except:
    print(movies[o], '(link not available)')

Users who liked Snow White and the Seven Dwarfs also liked: 
 
* Cinderella https://www.imdb.com/title/tt0042332/?ref_=fn_al_tt_1
* Pinocchio https://www.imdb.com/title/tt0032910/?ref_=fn_al_tt_1
* Beauty and the Beast https://www.imdb.com/title/tt0101414/?ref_=fn_al_tt_1
* Dumbo https://www.imdb.com/title/tt0033563/?ref_=fn_al_tt_1
* Fantasia https://www.imdb.com/title/tt0032455/?ref_=fn_al_tt_1
* Mary Poppins https://www.imdb.com/title/tt0058331/?ref_=fn_al_tt_1
* Aladdin https://www.imdb.com/title/tt0103639/?ref_=fn_al_tt_1
* Lion King, The (link not available)
* Alice in Wonderland https://www.imdb.com/title/tt0043274/?ref_=fn_al_tt_1
